In [206]:
import tqdm

%load_ext autoreload
%autoreload 2
import sys
import os
import wandb
import pandas as pd
import numpy as np
from plot_config import groups_to_plot, datasets_in_use, k_to_plot, list_of_list

sys.executable

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/eop/code/CORL/venv/bin/python'

In [207]:
from collections import defaultdict

api = wandb.Api(timeout=29)
scores = dict()

def write_scores(group, run, key_prefix):
    keys = [key_prefix + str(k) for k in k_to_plot]
    try:
        history = run.scan_history(keys=keys)
        for row in history:
            for k in k_to_plot:
                k = str(k)
                scores[group + k][run.config['_content']['env']][0].append(row[key_prefix + k])
    except KeyError:
        pass


for group in groups_to_plot:
    runs = api.runs(path="distill-llms/CORL", filters={"group": group})
    for k in k_to_plot:
        k = str(k)
        scores[group + k] = defaultdict(list_of_list)
        scores[group + "_inv_" + k] = defaultdict(list_of_list)
    for run in runs:
        write_scores(group, run, "eval/normalized_return_")
        write_scores(group + "_inv_", run, "eval/normalized_return_inv_")

scores


{'sweep100': defaultdict(<function plot_config.list_of_list()>,
             {'halfcheetah-medium-expert-v2': [[51.503008953438666,
                49.78987313816204,
                52.70902659526916,
                52.382153781487,
                50.30273537313863,
                52.50733440857787,
                51.919633902676885,
                51.0827841888744,
                53.41575825718853,
                52.009670870396654,
                49.81764278979793,
                50.39076455174808,
                50.69542150401396,
                50.10321736073355,
                49.876292641018054,
                51.574984457124096,
                51.08783811694977,
                51.72099282170183,
                51.828232150151834,
                49.273975983415646,
                52.2520994866484,
                48.72743526012983,
                52.12148706531657,
                52.26360858698417,
                53.02073047896813,
                49.6565266

In [208]:
for score in scores:
    for dataset in datasets_in_use:
        if dataset not in scores[score]:
            scores[score][dataset] = [[np.nan]]

scores

{'sweep100': defaultdict(<function plot_config.list_of_list()>,
             {'halfcheetah-medium-expert-v2': [[51.503008953438666,
                49.78987313816204,
                52.70902659526916,
                52.382153781487,
                50.30273537313863,
                52.50733440857787,
                51.919633902676885,
                51.0827841888744,
                53.41575825718853,
                52.009670870396654,
                49.81764278979793,
                50.39076455174808,
                50.69542150401396,
                50.10321736073355,
                49.876292641018054,
                51.574984457124096,
                51.08783811694977,
                51.72099282170183,
                51.828232150151834,
                49.273975983415646,
                52.2520994866484,
                48.72743526012983,
                52.12148706531657,
                52.26360858698417,
                53.02073047896813,
                49.6565266

In [209]:
import pickle


with open("bin/myscores_100.pkl", "wb") as f:
    pickle.dump(scores, f)